In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px

# --- 1. 데이터 임포트 ---
from services.tables.HR_Core.basic_info_table import emp_df
from services.tables.HR_Core.position_info_table import position_info_df
from services.tables.HR_Core.position_table import position_df, position_order, grade_order

def create_figure():
    """
    제안 4-3: 직위/직급별 경험 자산 현황 그래프를 생성합니다.
    """
    # --- 2. 데이터 준비 및 가공 ---
    current_emps_df = emp_df[emp_df['CURRENT_EMP_YN'] == 'Y'].copy()
    current_emps_df['TENURE_YEARS'] = current_emps_df['DURATION'] / 365.25

    # 현재 직위 및 직급 정보 추가
    current_positions = position_info_df[position_info_df['GRADE_END_DATE'].isnull()][['EMP_ID', 'POSITION_ID', 'GRADE_ID']]
    analysis_df = pd.merge(current_emps_df, current_positions, on='EMP_ID', how='left')
    analysis_df = pd.merge(analysis_df, position_df[['POSITION_ID', 'POSITION_NAME']].drop_duplicates(), on='POSITION_ID', how='left')
    
    analysis_df = analysis_df.dropna(subset=['POSITION_NAME', 'GRADE_ID', 'TENURE_YEARS'])

    # 근속년수 구간화 및 직위/직급별 집계
    analysis_df['TENURE_BIN'] = pd.cut(analysis_df['TENURE_YEARS'], bins=range(0, int(analysis_df['TENURE_YEARS'].max()) + 2), right=False, labels=range(0, int(analysis_df['TENURE_YEARS'].max()) + 1))
    pos_summary = analysis_df.groupby(['POSITION_NAME', 'TENURE_BIN'], observed=False).size().reset_index(name='COUNT')
    grade_summary = analysis_df.groupby(['POSITION_NAME', 'GRADE_ID', 'TENURE_BIN'], observed=False).size().reset_index(name='COUNT')

    # x축 범위 미리 계산
    x_max = analysis_df['TENURE_YEARS'].max()
    fixed_x_range = [-0.5, x_max + 1.5]

    # --- 3. Plotly 인터랙티브 그래프 생성 ---
    fig = go.Figure()
    colors = px.colors.qualitative.Plotly

    # 1. '전체' 뷰 트레이스 추가 (직위별 누적)
    for i, pos_name in enumerate(position_order):
        df_filtered = pos_summary[pos_summary['POSITION_NAME'] == pos_name]
        fig.add_trace(go.Bar(x=df_filtered['TENURE_BIN'], y=df_filtered['COUNT'], name=pos_name, marker_color=colors[i]))

    # 2. '상세' 뷰 트레이스 추가 (직급별 누적)
    grade_traces_map = {}
    trace_idx_counter = len(fig.data)
    for pos_name in position_order:
        grade_pos_df = grade_summary[grade_summary['POSITION_NAME'] == pos_name]
        grades_in_pos = [g for g in grade_order if g in grade_pos_df['GRADE_ID'].unique()]
        grade_traces_map[pos_name] = []
        for j, grade_id in enumerate(grades_in_pos):
            df_filtered = grade_pos_df[grade_pos_df['GRADE_ID'] == grade_id]
            fig.add_trace(go.Bar(
                x=df_filtered['TENURE_BIN'], y=df_filtered['COUNT'], name=grade_id,
                visible=False, marker_color=colors[j % len(colors)],
                showlegend=False
            ))
            grade_traces_map[pos_name].append(trace_idx_counter)
            trace_idx_counter += 1

    # --- 4. 드롭다운 메뉴 생성 ---
    buttons = []
    buttons.append(dict(label='전체', method='update', 
                        args=[{'visible': [True]*len(position_order) + [False]*(len(fig.data)-len(position_order))},
                              {'title': '전체 직위별 근속년수 분포', 'barmode': 'stack', 'showlegend': True, 'legend_title_text': '직위'}]))

    for pos_name in position_order:
        visibility_mask = [False] * len(fig.data)
        for trace_idx in grade_traces_map.get(pos_name, []):
            visibility_mask[trace_idx] = True
        buttons.append(dict(label=f'{pos_name}', method='update', 
                            args=[{'visible': visibility_mask},
                                  {'title': f'{pos_name} 내 직급별 근속년수 분포', 'barmode': 'stack', 'showlegend': False}]))

    # --- 5. 레이아웃 업데이트 및 그래프 표시 ---
    fig.update_layout(
        updatemenus=[dict(
            active=0, buttons=buttons, direction="down",
            pad={"r": 10, "t": 10}, showactive=True,
            x=0.01, xanchor="left", y=1.1, yanchor="top"
        )],
        title_text='직위/직급별 근속년수 분포 현황',
        xaxis_title='근속년수 (년)',
        yaxis_title='직원 수',
        font_size=14, height=700,
        bargap=0.2,
        barmode='stack',
        legend_title_text='직위',
        annotations=[dict(text="직위/직급 선택:", showarrow=False, x=0, y=1.08, yref="paper", align="left")],
        xaxis_range=fixed_x_range
    )
    fig.update_xaxes(dtick=1)
    
    return fig

# 이 파일을 직접 실행할 경우 그래프를 생성하여 보여줍니다.
pio.renderers.default = 'vscode'
fig = create_figure()
fig.show()